# Atomic column position identification
Here, a quick, simple method to identify atom column positions in atomic resolution HAADF STEM data is shown, as well as an example using these positions to visualize domains in PbTiO3. The techniques shown here are just a rough demonstration of how positions can be identified and used, for publication quality analysis much greater care is necessary. For further information about the data being used (and a more thorough domain mapping), see:

**"Ferroelectric Domain Walls in PbTiO3 Are Effective Regulators of Heat Flow at Room Temperature"**, Nano Lett. (2019), [DOI: 10.1021/acs.nanolett.9b02991](https://doi.org/10.1021/acs.nanolett.9b02991)

Other techniques/software packages for this type of analysis are available, e.g.:

https://github.com/quantitativeTEM/StatSTEM

https://atomap.org/

https://github.com/pycroscopy/atomai

To begin, run the cells below to load the required modules and define a gaussian to fit sites.

In [ ]:
# load modules
from photutils import DAOStarFinder
import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread, imwrite
from scipy import optimize
from time import time

%matplotlib notebook

# Loading and normalizing data

To avoid scan artifacts atom column finding should be performed on registered datasets with fast frame times.

Run the below cell to load a registered image of PbTiO3 on DSO. For interpretability, here **the image is normalized to have a minimum value of 0 and a maximum of 1.0**. Looking closely at the image you may be able to faintly see the domain structure by eye.

In [ ]:
f = "../Data/PTO_30_3,6Mx_130mm_BF_HAADF_DF4_21,4mrad_stack_1_corr_nosb.tif"
im = imread(f)

im = im - im.min()
im = im / im.max()

fig,ax = plt.subplots(1,1)
ax.matshow(im,cmap='gray')
ax.axis('off')

# <span style="color:red">Checkpoint:</span>
Try to identify the A sites (Pb) and B sites (Ti) in the image. Are the oxygens visible in this HAADF dataset? 

Is the film-substrate interface visible in this image? What is the growth direction?

Looking closely you may already be able to see by eye some kind of domain structure in the film -- can you tell the orientation of the boundaries?

# Quick position identification with DAOStarFinder
`DAOStarFinder`, developed for stellar photometry in crowded fields (https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract) does a good job of quickly identifying column positions with reasonable precision. However, parameters must be tuned to do a good job capturing most of the columns. Run the next cell to run the starfinder, and once it's done run the following one to plot all selected sites on the image, and generate a histogram of site amplitudes. Note that running the star finder will take some time, but should finish in less than a minute. 

Try experimenting with the <span style="color:red">threshold</span> and <span style="color:red">fwhm</span> to try to identify all columns in the image, only A sites (more intense columns), and only B sites (less intense columns). Once you understand how the parameters affect which sites are identified, pick a set of parameters that identify as many sites as possible (both A and B sites) without identifying any positions not on atomic columns, and proceed to the next set of cells. 

In the intensity histogram there should be two well defined peaks, and at least 7000 positions should be identified, although some sites will probably still be missing. This is fine for a quick visualization, but if getting every site is necessary additional techniques such as background subtraction can be used to improve identification.

Hints: image intensity was normalized to lie between 0 and 1, so the threshold should be towards the low end of that range. Inspect the displayed image above to get an idea of how many pixels across an atom is to help pick the fwhm -- something in the range 5-25 is reasonable.

In [ ]:
threshold = ??  #minimum intensity to identify
fwhm =  ??      #full width half maximum of atoms, can estiamte from inspecting image


t0=time()
DAO = DAOStarFinder(threshold, fwhm)
res = DAO.find_stars(im)

x = np.array(res.field(1)) # atom x positions in pixels
y = np.array(res.field(2)) # atom y positions in pixels
amp = res.field(8)         # atom intensity maxima

n_sites = len(x)
t = time()-t0
print(f"Identified {n_sites} sites in {t%60} seconds")



In [ ]:
# check position identification. Can use the box zoom tool below the figure to zoom in closely to the image
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax[0].matshow(im,cmap='gray')
ax[0].plot(x,y,'r.')
ax[0].axis('off')
ax[0].set_title("Identified positions")
_ = ax[1].hist(amp,bins=100)
ax[1].set_title('Site Intensities')
ax[1].set_xlabel('Intensity (arbitrary unit)')
ax[1].set_ylabel('Count')

# <span style="color:red">Checkpoint:</span>
Compare your identified positions with your group -- discuss the effect of the threshold and fwhm parameters. Did some parameter choices seem to do a better job selecting A sites, and others do a better job of selecting B sites? If so which ones?

Can you think of a multi-pass technique that might do a better job identifiying all positions? What considerations must be made to avoid complications such as double counting?

For this tutorial, for your final set of parameters stick with something that seemed to do a reasonable job identifying both sites.

## Segmenting A/B sites
In this structure A and B sites can be pretty easily distinguished by filtering based on peak intensity. Using the histogram above, identify the indices that correspond to A and B sites by thresholding the <span style="color:orange">amp</span> variable so that the **high amplitude sites are A sites** and the **low amplitude sites are B sites** are seperated. Do this by choosing a value for the  <span style="color:red">amplitude_threshold</span> variable.

In [ ]:
amplitude_threshold = ??


indexes_A = (amp > amplitude_threshold)
indexes_B = (amp < amplitude_threshold)* (amp > 0.0)
fig,ax = plt.subplots(1,1)
ax.matshow(im,cmap='gray')
ax.plot(x[indexes_A],y[indexes_A],'r.')
ax.plot(x[indexes_B],y[indexes_B],'b.')
ax.axis('off')


# <span style="color:red">Checkpoint:</span>
Zoom in and look closely at the sites identified as A sites vs. B sites. Are there many misidentifications? Which do you think will contribute more to the final error in column identification / domain mapping, incorrect column finding with the star finder or mislabeling of columns with this intensity based segmentation?

## Identify domains in B-site displacements
The ferroelectric domains in the structure can be identified by looking at the displacement of the B site compared to the 4 surrounding A sites. Calculate the offset of each B site versus the center of the 4 surrounding A sites.

In [ ]:
# takes 1-3 minutes, continue to the checkpoint below while it runs

def dists_to_pt(ptx,pty,xs,ys):
    dists = np.zeros(xs.shape)
    for it in range(len(xs)):
        dists[it] = np.sqrt((ptx-xs[it])**2+(pty-ys[it])**2)
    return dists

def closest_pt(ptx,pty,xs,ys,n=0):
    dists = dists_to_pt(ptx,pty,xs,ys)
    sorts = np.argsort(dists)
    return sorts[:n]

xB = x[indexes_B].copy()
yB = y[indexes_B].copy()
xA = x[indexes_A].copy()
yA = y[indexes_A].copy()

threshold = 30 #identify bad offsets --  set to NaN

offset = np.zeros((len(xB),2))
for itB in range(len(xB)):
    c4 = closest_pt(xB[itB],yB[itB],xA,yA,n=4)
    offsets = np.zeros((2,4))
    for ito in range(4):
        offsets[:,ito] =(xB[itB]-xA[c4[ito]],yB[itB]-yA[c4[ito]])
    offset[itB] = np.sum(offsets,1)
    if offset[itB,0]*offset[itB,0]+offset[itB,1]*offset[itB,1] >threshold*threshold :
        offset[itB,:] = (np.nan,np.nan) 

# B sites which didn't meet threshold will be NaN
angles = np.arctan2(offset[:,0],offset[:,1])

# <span style="color:red">Checkpoint:</span>
Look through the code in the cell above while it is running -- are you able to follow what is being done? Discuss the algorithm with your group. How are errors in column identification mitigated? How might this affect the results, and  how might this approach be improved / made more robust to column identification errors?

## Visualize domains
Plots arrows corresponding to the offsets of each B site, the colors correspond to the angle of the offset vector, giving a rough visual of the domain structure. Zoom in to better see the offsets. Some of the displacements are very large and at seemingly incorrect angles, other sites are missing offset vectors altogether.

In [ ]:
# shift angles to take advantage of colormap
angles_map = angles.copy()
angles_map_shift = 4 #adjust to shift angle colormap
angles_map = np.mod(-angles_map+np.pi+angles_map_shift,2*np.pi)

fig,ax = plt.subplots(1,1)
ax.matshow(im,cmap='gray')
q = ax.quiver(xB, yB, offset[:,0], offset[:,1], angles_map,cmap='hsv',units='xy',angles='xy')
fig.colorbar(q, ax=ax,label='Angle (rad.)')
# 13 pm/px, 
ax.quiverkey(Q=q,X=.5, Y=-.05,U=7.7,label="1 Å",labelpos='E',coordinates='axes')
ax.axis('off')

# <span style="color:red">Checkpoint:</span>
Compare your result with your group members and the TA's solution. Were some mappings better than others? Why might this be the case / what parameter selections seemed to yield the best results? Try to explore the parameter space to get the best mapping possible. 

What is the largest source of error in the mapping, and how might it be addressed? 

Two common techniques used when working with atomic column positions are (1) refinement of atomic column positions with Gaussian fits to improve position precision, and (2) image preprocessing (filtering, background subtraction, thresholding, etc.) to improve site identification. Which of these might be more useful for addressing the errors encountered mapping domains in this dataset?